Notebook created: 2018-05-23 03:37:56  
Generated from: _build_jl/jl/need_for_speed.rst  

```asm
    pushq   %rbp
    movq    %rsp, %rbp
    addq    %rdi, %rdi
    leaq    (%rdi,%rsi,8), %rax
    popq    %rbp
    retq
    nopl    (%rax)
```


In [1]:
function f(a, b)
    y = 2a + 8b
    return y
end

f (generic function with 1 method)

```python
def f(a, b):
    y = 2 * a + 8 * b
    return y
```


```c
int f(int a, int b) {
    int y = 2 * a + 8 * b;
    return y;
}
```


In [2]:
function f(a, b)
    y = (a + 8b)^2
    return 7y
end

f (generic function with 1 method)

In [ ]:
@code_native f(1, 2)

```none
pushq   %rbp
movq    %rsp, %rbp
leaq    (%rdi,%rsi,8), %rdi
movabsq $power_by_squaring, %rax
movl    $2, %esi
callq   %rax
imulq   $7, %rax, %rax
popq    %rbp
retq
nop
```


In [ ]:
@code_native f(1.0, 2.0)

```none
pushq   %rbp
movq    %rsp, %rbp
movabsq $139613711993752, %rax  # imm = 0x7EFA59B58B98
mulsd   (%rax), %xmm1
addsd   %xmm0, %xmm1
mulsd   %xmm1, %xmm1
movabsq $139613711993760, %rax  # imm = 0x7EFA59B58BA0
mulsd   (%rax), %xmm1
movapd  %xmm1, %xmm0
popq    %rbp
retq
nop
```


## global variable 

In [3]:
b = 1.0
function g(a)
    for i in 1:1_000_000
        tmp = a + b
    end
end

g (generic function with 1 method)

In [ ]:
@time g(1.0)

```none
0.023039 seconds (2.00 M allocations: 30.518 MB, 12.90% gc time)
```


In [ ]:
@code_native g(1.0)

```none
pushq   %rbp
movq    %rsp, %rbp
pushq   %r15
pushq   %r14
pushq   %r13
pushq   %r12
pushq   %rbx
subq    $56, %rsp
movsd   %xmm0, -88(%rbp)
movq    %fs:0, %r15
addq    $-2672, %r15            # imm = 0xFFFFFFFFFFFFF590
xorpd   %xmm0, %xmm0
movupd  %xmm0, -64(%rbp)
movq    $0, -48(%rbp)
movq    $6, -80(%rbp)
movq    (%r15), %rax
movq    %rax, -72(%rbp)
leaq    -80(%rbp), %rax
movq    %rax, (%r15)
movabsq $140242067578672, %r12  # imm = 0x7F8CA69EDF30
movl    $1000000, %ebx          # imm = 0xF4240
leaq    5596992(%r12), %r13
movabsq $jl_apply_generic, %r14
nop
movq    75966904(%r12), %rax
movq    %rax, -48(%rbp)
movq    %r13, -64(%rbp)
movl    $1432, %esi             # imm = 0x598
movl    $16, %edx
movq    %r15, %rdi
movabsq $jl_gc_pool_alloc, %rax
callq   %rax
movq    %r12, -8(%rax)
movsd   -88(%rbp), %xmm0        # xmm0 = mem[0],zero
movsd   %xmm0, (%rax)
movq    %rax, -56(%rbp)
movl    $3, %esi
leaq    -64(%rbp), %rdi
callq   %r14
decq    %rbx
jne     L112
movq    -72(%rbp), %rax
movq    %rax, (%r15)
addq    $56, %rsp
popq    %rbx
popq    %r12
popq    %r13
popq    %r14
popq    %r15
popq    %rbp
retq
nopw    %cs:(%rax,%rax)
```


In [4]:
function g(a, b)
    for i in 1:1_000_000
        tmp = a + b
    end
end

g (generic function with 2 methods)

In [ ]:
@time g(1.0, 1.0)

```none
0.002876 seconds (1.31 k allocations: 61.374 KB)
```


In [5]:
@time g(1.0, 1.0)

  0.004688 seconds (1.09 k allocations: 58.673 KiB)


```none
0.000001 seconds (4 allocations: 160 bytes)
```


In [ ]:
@code_native g(1.0, 1.0)

```none
pushq   %rbp
movq    %rsp, %rbp
popq    %rbp
retq
nopw    %cs:(%rax,%rax)
```


another way of maintaining global variable

In [6]:
const b_const = 1.0
function g(a)
    for i in 1:1_000_000
        tmp = a + b_const
    end
end

g (generic function with 2 methods)

In [7]:
@code_native g(1.0)

	.text
Filename: In[6]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 4
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


In [8]:
@time g(1.0)

  0.000028 seconds (5 allocations: 240 bytes)


untyped case

In [9]:
struct Foo_generic
    a
end

ab abstract type on the field a

In [12]:
struct Foo_abstract
    a::Real
end

, here’s the parametrically typed case

In [21]:
struct Foo_concrete{T <: Real}
    a::T
end

In [13]:
fg = Foo_generic(1.0)
fa = Foo_abstract(1.0)
fc = Foo_concrete(1.0)

Foo_concrete{Float64}(1.0)

In [16]:
typeof(fa)

Foo_abstract

In [15]:
typeof(fg)

Foo_generic

In [22]:
typeof(fc)
fc.a

1.0

```none
Foo_concrete{Float64}
```


## timing

In [18]:
function f(foo)
    for i in 1:1_000_000
        tmp = i + foo.a
    end
end

f (generic function with 2 methods)

In [19]:
@time f(fg)

  0.049348 seconds (2.00 M allocations: 30.581 MiB, 22.06% gc time)


```none
0.029499 seconds (2.00 M allocations: 30.510 MB, 21.81% gc time)
```


In [ ]:
@code_native f(fg)

```none
pushq   %rbp
movq    %rsp, %rbp
pushq   %r15
pushq   %r14
pushq   %r13
pushq   %r12
pushq   %rbx
subq    $56, %rsp
movq    %rdi, %r15
movq    %fs:0, %rcx
addq    $-2672, %rcx            # imm = 0xFFFFFFFFFFFFF590
movq    %rcx, -88(%rbp)
xorps   %xmm0, %xmm0
movups  %xmm0, -64(%rbp)
movq    $0, -48(%rbp)
movq    $6, -80(%rbp)
movq    (%rcx), %rax
movq    %rax, -72(%rbp)
leaq    -80(%rbp), %rax
movq    %rax, (%rcx)
movl    $1, %ebx
movabsq $140242073175664, %r13  # imm = 0x7F8CA6F44670
movabsq $jl_box_int64, %r12
movabsq $jl_apply_generic, %r14
movq    (%r15), %rax
movq    %rax, -48(%rbp)
movq    %r13, -64(%rbp)
movq    %rbx, %rdi
leaq    1(%rbx), %rbx
callq   %r12
movq    %rax, -56(%rbp)
movl    $3, %esi
leaq    -64(%rbp), %rdi
callq   %r14
cmpq    $1000001, %rbx          # imm = 0xF4241
jne     L112
movq    -72(%rbp), %rax
movq    -88(%rbp), %rcx
movq    %rax, (%rcx)
addq    $56, %rsp
popq    %rbx
popq    %r12
popq    %r13
popq    %r14
popq    %r15
popq    %rbp
retq
nopl    (%rax,%rax)
```


In [ ]:
@time f(fa)

```none
0.030892 seconds (2.00 M allocations: 30.585 MB, 7.18% gc time)
```


In [ ]:
@time f(fc)

```none
0.002850 seconds (1.45 k allocations: 67.642 KB)
```


```none
0.000001 seconds (3 allocations: 144 bytes)
```


In [ ]:
@code_native f(fc)

```none
pushq   %rbp
movq    %rsp, %rbp
popq    %rbp
retq
nopw    %cs:(%rax,%rax)
```


## abstract container

In [27]:
function sum_float_array(x::Array{Float64, 1})
    sum = 0.0
    for i in 1:length(x)
        sum += x[i]
    end
    return sum
end

sum_float_array (generic function with 1 method)

In [28]:
x=linspace(0,1,1e6)

0.0:1.000001000001e-6:1.0

In [29]:
x = linspace(0, 1, 1e6)
x = collect(x)
typeof(x)

Array{Float64,1}

```none
Array{Float64,1}
```


In [30]:
@time sum_float_array(x)

  0.006840 seconds (1.17 k allocations: 62.093 KiB)


499999.9999999796

```none
0.005524 seconds (1.74 k allocations: 82.486 KB)
```


## type inference

In [31]:
function sum_array(x)
    sum = 0.0
    for i in 1:length(x)
        sum += x[i]
    end
    return sum
end

sum_array (generic function with 1 method)

In [32]:
@time sum_array(x)

  0.007184 seconds (1.17 k allocations: 62.171 KiB)


499999.9999999796

```none
0.005556 seconds (1.61 k allocations: 75.002 KB)
```


## an abstract container

In [33]:
x = Any[1/i for i in 1:1e6];

In [34]:
eltype(x)

Any

```none
Any
```


In [35]:
@time sum_array(x)

  0.049881 seconds (1.00 M allocations: 15.319 MiB, 43.10% gc time)


14.392726722864989

```none
0.021680 seconds (1.00 M allocations: 15.332 MB)
```
